### Section bodies combined by position order top 1000 words

#### Steps:

1. Get the relevant sections and their position in the report
2. Order the sections by position
3. Extract the body of section and keep on concatenating
4. Extract top 1000 words
5. If no relevant sections -> Write the complete file as summary with top 1000 words
6. Compare the system summary with each gold summary and get the rouge score. Take average of all gold summaries for the respective file.
7. Take average of all the scores for all the files.

In [1]:
import pandas as pd
import pickle
import os

from extract_section_body import extract_section_body
from rouge_evaluation import get_rouge_scores

In [2]:
VALIDATION_DATASET = True 
TEST_DATASET = False

In [3]:
if VALIDATION_DATASET:
    dir_ = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/'
    toc_loc_pkl_file_path = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/out/valid_toc_loc.pkl'
    df_predicted_path = 'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/2_Section_Classification/out/validation_df_predicted.pkl'

if TEST_DATASET:
    dir_ = '../../../Dataset/FNS2022/English/testing/'
    toc_loc_pkl_file_path = '../../../Dataset/Annotated_Dataset/test_toc_loc.pkl'
    df_predicted_path = '../../FNP2022/2_Section_Classification/out/test_df_predicted.pkl'

annual_reports_dir = "annual_reports"
gold_summary_dir = "gold_summaries"
system_summary_dir = 'section_combined_by_pos_top_1000'
team_name = 'IT-356'
dir_

'C:/My folders/Deva paul/AI/SEM 5/356 NLP/NLP Project/New folder/FNS2023_Datasets/English/validation/'

In [4]:
df_predicted = pickle.load(open(df_predicted_path, 'rb'))
df_predicted

,file_id,toc_section,toc_section_pos,toc_section_len,is_section_in_summary,toc_section_cleaned,pred,False,True
0,30777,Financial and operational highlights,161,22,0,financi oper highlight,1,0.407563,0.592437
1,30777,Strategic report,183,6,1,strateg report,0,0.932972,0.067028
2,30777,Global network,189,11,0,global network,0,0.756613,0.243387
3,30777,Chairman’s statement,200,4,1,chairman statement,1,0.018573,0.981427
4,30777,Chief Executive’s review,204,4,1,chief execut review,1,0.007140,0.992860
...,...,...,...,...,...,...,...,...,...
10547,4162,"to 110,",21914,1104,0,,0,0.983343,0.016657
10548,4162,and 117,23018,1104,0,,0,0.985658,0.014342
10549,4162,to 116,24122,1104,0,,0,0.987655,0.012345
10550,4162,to 122,25226,5440,0,,0,0.987178,0.012822


In [5]:
def get_relevant_sections_with_score(file_id):
    result_dict = df_predicted[(df_predicted.file_id == int(file_id)) & (df_predicted.pred == 1)][['toc_section', 'True']].to_dict('list')
    return result_dict

In [18]:
file_id=4149
relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        # Section order is maintained
relevant_sections = relevant_sections_with_score['toc_section']
print(file_id, relevant_sections)
summary = ""
if relevant_sections:
    for section in relevant_sections:
        summary = summary + " " +extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
    summary_split = summary.split(' ')
    number_of_words = len(summary_split)
    print(number_of_words)
else:
    print('Relevant Section Not Found in ', file_id)
    summary = open(os.path.join(dir_, annual_reports_dir, '4117.txt'), "r", encoding="utf-8").read()
    summary_split = summary.split(' ')
    number_of_words = len(summary_split)
    print(number_of_words)

4149 ['start', 'At a glance', 'Chief Executives review']
6405


In [15]:
num_file = 0
os.makedirs(system_summary_dir)
for file in os.listdir(os.path.join(dir_, annual_reports_dir)):
    try:
        if num_file%50 == 0:
            print("Processing File Number: ", num_file)
        num_file = num_file +1 
        file_id = file.split('.')[0]
        relevant_sections_with_score = get_relevant_sections_with_score(file_id)
        # Section order is maintained
        relevant_sections = relevant_sections_with_score['toc_section']
        #print(file_id, relevant_sections)
        summary = ""
        if relevant_sections:
            x=0
            #print('Relevant Section Found in ', file_id)
            for section in relevant_sections:
                summary = summary + " " +extract_section_body(file_id, section, dir_, annual_reports_dir, toc_loc_pkl_file_path)
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            x+=number_of_words
            #print(number_of_words)
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
            print(f"Number of words in narrative sections of {file_id}: ",x)
        else:
            #print('Relevant Section Not Found in ', file_id)
            summary = open(os.path.join(dir_, annual_reports_dir, file), "r", encoding="utf-8").read()
            summary_split = summary.split(' ')
            number_of_words = len(summary_split)
            #print(number_of_words)
            if number_of_words > 1000:
                summary = " ".join(summary_split[:1000])
        
        with open(os.path.join(system_summary_dir, file_id+'_'+team_name+'.txt'), 'w', encoding='utf-8') as f:
            f.write(str(summary))
            
        if ".DS_Store" in file:
            continue
    except Exception as e:
        print(file, e)

Processing File Number:  0
Number of words in narrative sections of 30777:  169
Number of words in narrative sections of 30778:  6360
Number of words in narrative sections of 30779:  8215
Number of words in narrative sections of 30781:  2872
Number of words in narrative sections of 30782:  6182
Number of words in narrative sections of 30783:  1615
Number of words in narrative sections of 30785:  4755
Number of words in narrative sections of 30790:  2969
Number of words in narrative sections of 30795:  7823
Number of words in narrative sections of 30796:  8315
Number of words in narrative sections of 30800:  1798
Number of words in narrative sections of 30808:  7086
Number of words in narrative sections of 30813:  1968
Number of words in narrative sections of 30815:  9545
Number of words in narrative sections of 30816:  1711
Number of words in narrative sections of 30817:  2303
Number of words in narrative sections of 30819:  5007
Number of words in narrative sections of 30820:  9069
Nu

Number of words in narrative sections of 31623:  2974
Number of words in narrative sections of 31628:  2873
Number of words in narrative sections of 31631:  6440
Number of words in narrative sections of 31632:  4834
Number of words in narrative sections of 31633:  3269
Number of words in narrative sections of 31634:  2236
Number of words in narrative sections of 31646:  3866
Number of words in narrative sections of 31648:  5948
Number of words in narrative sections of 31650:  4625
Number of words in narrative sections of 31654:  3195
Number of words in narrative sections of 31655:  6477
Number of words in narrative sections of 31659:  2449
Number of words in narrative sections of 31661:  4363
Number of words in narrative sections of 31664:  620
Number of words in narrative sections of 31678:  2797
Number of words in narrative sections of 31681:  6691
Number of words in narrative sections of 31685:  3872
Number of words in narrative sections of 31699:  2481
Number of words in narrative 

Number of words in narrative sections of 32602:  4862
Number of words in narrative sections of 32609:  5529
Number of words in narrative sections of 32616:  3742
Number of words in narrative sections of 32623:  518
Number of words in narrative sections of 32648:  4147
Number of words in narrative sections of 32651:  6741
Number of words in narrative sections of 32657:  1650
Number of words in narrative sections of 32684:  3726
Number of words in narrative sections of 32688:  3158
Number of words in narrative sections of 32693:  4709
Number of words in narrative sections of 32717:  1655
Number of words in narrative sections of 32734:  1558
Number of words in narrative sections of 32771:  3941
Number of words in narrative sections of 32773:  1755
Number of words in narrative sections of 32792:  4432
Number of words in narrative sections of 32809:  2731
Number of words in narrative sections of 32810:  9621
Number of words in narrative sections of 32825:  3169
Number of words in narrative 

In [16]:
if VALIDATION_DATASET:
    gold_summary_dir_ =  os.path.join(dir_, gold_summary_dir)
    rouge_scores = get_rouge_scores(system_summary_dir, gold_summary_dir_)
    print(rouge_scores)

Processing File Number:  0
Processing File Number:  50
Processing File Number:  100
Processing File Number:  150
Processing File Number:  200
Processing File Number:  250
Processing File Number:  300
Processing File Number:  350
Processing File Number:  400
Number of files processed:  413
{'rouge-1': {'p': 0.5285770806941444, 'r': 0.5545217748693185, 'f': 0.46361306271599767}, 'rouge-2': {'p': 0.32989796430454793, 'r': 0.4051547508010742, 'f': 0.3152998505961532}}
